In [1]:
from dataclasses import dataclass

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.firefox.options import Options


@dataclass
class Credentials:
    """
    максимально небезопасно хранить
    логин и пароль в коде
    """

    username: str = "admin"
    password: str = "zh7cqc8v"
    domain: str = "997267.34.rt.ru"


class CallsParser:

    """
    парсер страницы контакт центр
    https://p1.cloudpbx.rt.ru/callcenter/reports/
    """

    def __init__(
        self,
        date_number: int,
        credentials: Credentials,
        path_to_driver: str = "./geckodriver.exe",
    ) -> None:
        assert date_number in range(1, 32), "Неверный день месяца"
        assert credentials.username, "Не указан логин"
        assert credentials.password, "Не указан пароль"

        self.date_number = date_number
        self.path_to_driver = path_to_driver

        self.service = Service(self.path_to_driver)
        driver_options = Options()
        # driver_options.add_argument('--proxy-server=10.10.36.22:3128')
        
        self.driver = webdriver.Firefox(
            service=self.service,
            options=driver_options,
        )

        self.initial_web_page = "https://cloudpbx.rt.ru/callcenter/reports/"
        self.credentials = credentials

    def parse(self) -> None:
        self.driver.get(self.initial_web_page)

        username = self.driver.find_element(
            by=By.ID,
            value="username",
        )
        password = self.driver.find_element(
            by=By.ID,
            value="password",
        )
        domain = self.driver.find_element(
            by=By.ID,
            value="domain",
        )

        signin_button = self.driver.find_element(
            by=By.CLASS_NAME,
            value="btn.btn-lg.orange.mb-3.waves-effect.waves-light",
        )

        username.send_keys(self.credentials.username)
        password.send_keys(self.credentials.password)
        domain.send_keys(self.credentials.domain)
        signin_button.click()

        WebDriverWait(self.driver, timeout=30).until(
            lambda d: d.find_element(
                by=By.XPATH,
                value="//a[@href='/callcenter/']",
            ),
        )

        item_1 = self.driver.find_element(
            by=By.XPATH,
            value="//a[@href='/callcenter/']",
        )

        item_1.click()

        item_2 = self.driver.find_element(
            by=By.XPATH,
            value="//a[@href='/callcenter/reports/']",
        )
        item_2.click()

        WebDriverWait(self.driver, timeout=30).until(
            lambda d: d.find_element(
                by=By.XPATH,
                value="//select[@name='report_type_id']",
            ),
        )

        item_3 = self.driver.find_element(
            by=By.XPATH,
            value="//select[@name='report_type_id']",
        )
        item_3.click()

        item_4 = item_3.find_element(
            by=By.XPATH,
            value="//option[@value='7']",
        )

        item_4.click()

        WebDriverWait(self.driver, timeout=10).until(
            lambda d: d.find_element(
                by=By.CLASS_NAME,
                value="ui-datepicker-trigger",
            )
        )

        self.driver.execute_script(
            "document.querySelector('.ui-datepicker-trigger').click();"
        )

        TARGET_DATE = 23

        self.driver.execute_script(
            f"""
            let _$ = (selector) => document.querySelector(selector);
            let _$$ = (selector) => document.querySelectorAll(selector);
            
            _$('.ui-datepicker-calendar')
            .querySelectorAll(".ui-state-default")[{TARGET_DATE - 1}]
            .click();
            _$(".ui-datepicker-close.ui-state-default.ui-priority-primary.ui-corner-all").click();
            
            _$$('.ui-datepicker-trigger')[1].click();
            _$('.ui-datepicker-calendar')
            .querySelectorAll(".ui-state-default")[{TARGET_DATE - 1}].click();
            """
        )

        slider_1 = self.driver.find_element(
            by=By.ID,
            value="hourSlider",
        )
        slider_2 = self.driver.find_element(
            by=By.ID,
            value="minuteSlider",
        )
        button_1 = slider_1.find_element(
            by=By.CLASS_NAME,
            value="ui-slider-handle.ui-corner-all.ui-state-default",
        )
        action_1 = webdriver.ActionChains(self.driver)
        action_1.move_to_element(slider_1).click().perform()
        action_1.move_to_element(slider_1).click_and_hold(button_1).move_by_offset(
            0, -slider_1.size["height"] / 2
        ).click().perform()

        button_2 = slider_2.find_element(
            by=By.CLASS_NAME,
            value="ui-slider-handle.ui-corner-all.ui-state-default",
        )
        action_2 = webdriver.ActionChains(self.driver)
        action_2.move_to_element(slider_2).click().perform()
        action_2.move_to_element(slider_2).click_and_hold(button_2).move_by_offset(
            0, -slider_2.size["height"] / 2
        ).click().perform()

        close_button = self.driver.find_element(
            by=By.CLASS_NAME,
            value="ui-datepicker-close.ui-state-default.ui-priority-primary.ui-corner-all",
        )

        close_button.click()

        download_button = self.driver.find_element(
            by=By.CLASS_NAME,
            value="save",
        )

        download_button.click()
        self.driver.close()  # close the current window


if __name__ == "__main__":
    credentials = Credentials(
        username="admin",
        password="zh7cqc8v",
        domain="997267.34.rt.ru",
    )
    parser = CallsParser(date_number=23, credentials=credentials)
    parser.parse()

WebDriverException: Message: <!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 4.01 Transitional//EN" "http://www.w3.org/TR/html4/loose.dtd">
<HTML><HEAD><META HTTP-EQUIV="Content-Type" CONTENT="text/html; charset=iso-8859-1">
<TITLE>ERROR: The requested URL could not be retrieved</TITLE>
<STYLE type="text/css"><!--BODY{background-color:#ffffff;font-family:verdana,sans-serif}PRE{font-family:sans-serif}--></STYLE>
</HEAD><BODY>
<H1>ERROR</H1>
<H2>The requested URL could not be retrieved</H2>
<HR noshade size="1px">
<P>
While trying to retrieve the URL:
<A HREF="http://localhost:59130/session">http://localhost:59130/session</A>
<P>
The following error was encountered:
<UL>
<LI>
<STRONG>
Connection to 127.0.0.1 Failed
</STRONG>
</UL>

<P>
The system returned:
<PRE><I>    (61) Connection refused</I></PRE>

<P>
The remote host or network may be down.  Please try the request again.
<P>Your cache administrator is <A HREF="mailto:webmaster">webmaster</A>. 

<BR clear="all">
<HR noshade size="1px">
<ADDR